## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file.
file_to_open = os.path.join("..", "Weather_Database", "WeatherPY_Database.csv")
city_data_df = pd.read_csv(file_to_open)
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description
0,0,Punta Arenas,-53.1500,-70.9167,48.31,71,75,8.05,CL,2022-02-02 22:50:20,broken clouds
1,1,Berlevag,70.8578,29.0864,26.85,64,100,11.88,NO,2022-02-02 22:51:56,light snow
2,2,Ponta Do Sol,32.6667,-17.1000,67.01,77,100,6.08,PT,2022-02-02 22:49:14,overcast clouds
3,3,Bilibino,68.0546,166.4372,-6.84,63,49,4.90,RU,2022-02-02 22:51:57,scattered clouds
4,4,Carnarvon,-24.8667,113.6333,77.07,88,67,11.50,AU,2022-02-02 22:50:11,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input(f'What is minimum temp?'))
max_temp = float(input(f'What is maximum temp?'))

What is minimum temp?70
What is maximum temp?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) &
                               (city_data_df['Max Temp'] <= max_temp)]

preferred_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description
4,4,Carnarvon,-24.8667,113.6333,77.07,88,67,11.50,AU,2022-02-02 22:50:11,broken clouds
5,5,Lorengau,-2.0226,147.2712,80.26,80,100,11.54,PG,2022-02-02 22:48:56,light rain
8,8,Avarua,-21.2078,-159.7750,84.25,79,40,18.41,CK,2022-02-02 22:49:25,scattered clouds
20,20,Lokoja,7.8024,6.7430,79.29,18,9,2.75,NG,2022-02-02 22:52:01,clear sky
27,27,Ahipara,-35.1667,173.1667,75.31,68,100,13.33,NZ,2022-02-02 22:52:03,overcast clouds
...,...,...,...,...,...,...,...,...,...,...,...
687,687,Honiara,-9.4333,159.9500,83.64,78,100,6.29,SB,2022-02-02 23:08:49,overcast clouds
690,690,Lahaina,20.8783,-156.6825,83.44,72,0,2.75,US,2022-02-02 23:09:57,clear sky
692,692,Tapes,-30.6733,-51.3958,76.26,83,0,16.53,BR,2022-02-02 23:09:58,clear sky
694,694,Sabang,5.8933,95.3214,80.26,79,42,9.57,ID,2022-02-02 23:10:59,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_df.isnull().sum()

City_ID        0
City           0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Country        1
Date           0
Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_df = preferred_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
4,Carnarvon,AU,77.07,broken clouds,-24.8667,113.6333,
5,Lorengau,PG,80.26,light rain,-2.0226,147.2712,
8,Avarua,CK,84.25,scattered clouds,-21.2078,-159.7750,
20,Lokoja,NG,79.29,clear sky,7.8024,6.7430,
27,Ahipara,NZ,75.31,overcast clouds,-35.1667,173.1667,
32,Rikitea,PF,78.66,overcast clouds,-23.1203,-134.9692,
34,Mar Del Plata,AR,78.82,clear sky,-38.0023,-57.5575,
38,Alta Floresta,BR,75.02,overcast clouds,-9.8756,-56.0861,
44,Waipawa,NZ,74.03,overcast clouds,-41.4122,175.5153,
45,Port Blair,IN,77.04,mist,11.6667,92.7500,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index,row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat= row['Lat']
    lng= row['Lng']
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print('Hotel not found.....skipping') 

Hotel not found.....skipping
Hotel not found.....skipping
Hotel not found.....skipping
Hotel not found.....skipping
Hotel not found.....skipping
Hotel not found.....skipping
Hotel not found.....skipping
Hotel not found.....skipping
Hotel not found.....skipping
Hotel not found.....skipping
Hotel not found.....skipping
Hotel not found.....skipping
Hotel not found.....skipping
Hotel not found.....skipping
Hotel not found.....skipping
Hotel not found.....skipping
Hotel not found.....skipping


In [16]:
# Check for NaN values
hotel_df.isnull().sum()


City           0
Country        0
Max Temp       0
Description    0
Lat            0
Lng            0
Hotel Name     0
dtype: int64

In [19]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
4,Carnarvon,AU,77.07,broken clouds,-24.8667,113.6333,Hospitality Carnarvon
5,Lorengau,PG,80.26,light rain,-2.0226,147.2712,Lorengau Harbourside Hotel
8,Avarua,CK,84.25,scattered clouds,-21.2078,-159.7750,Paradise Inn
20,Lokoja,NG,79.29,clear sky,7.8024,6.7430,Hotel Charvid
27,Ahipara,NZ,75.31,overcast clouds,-35.1667,173.1667,Ahipara Holiday Park


In [20]:
# 8a. Create the output File (CSV)
output_data_file = os.path.join("../Vacation_Search/WeatherPy_vacation.csv")
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [22]:
# 11a. Add a marker layer for each city to the map. 
max_temps = hotel_df['Max Temp']

fig = gmaps.figure(center=(30.0,31), zoom_level= 1.5)
heat_layer= gmaps.heatmap_layer(locations, weights=max_temps, dissipating= False, max_intensity= 300, point_radius= 4)
marker_layer= gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig.add_layer(heat_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))